In [ ]:
#import libraries
import pandas as pd
import numpy as np
from sklearn import model_selection
from sklearn import metrics
from sklearn import preprocessing
from scipy import sparse
from sklearn import decomposition
from sklearn import ensemble
import random

import warnings
warnings.simplefilter("ignore", UserWarning)

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Datasets/train.csv')

In [ ]:
df

,id,bin_0,bin_1,bin_2,bin_3,bin_4,nom_0,nom_1,nom_2,nom_3,nom_4,nom_5,nom_6,nom_7,nom_8,nom_9,ord_0,ord_1,ord_2,ord_3,ord_4,ord_5,day,month,target
0,0,0.0,0.0,0.0,F,N,Red,Trapezoid,Hamster,Russia,Bassoon,de4c57ee2,a64bc7ddf,598080a91,0256c7a4b,02e7c8990,3.0,Contributor,Hot,c,U,Pw,6.0,3.0,0
1,1,1.0,1.0,0.0,F,Y,Red,Star,Axolotl,NaN,Theremin,2bb3c3e5c,3a3a936e8,1dddb8473,52ead350c,f37df64af,3.0,Grandmaster,Warm,e,X,pE,7.0,7.0,0
2,2,0.0,1.0,0.0,F,N,Red,NaN,Hamster,Canada,Bassoon,b574c9841,708248125,5ddc9a726,745b909d1,NaN,3.0,NaN,Freezing,n,P,eN,5.0,9.0,0
3,3,NaN,0.0,0.0,F,N,Red,Circle,Hamster,Finland,Theremin,673bdf1f6,23edb8da3,3a33ef960,bdaa56dd1,f9d456e57,1.0,Novice,Lava Hot,a,C,NaN,3.0,3.0,0
4,4,0.0,NaN,0.0,T,N,Red,Triangle,Hamster,Costa Rica,NaN,777d1ac2c,3a7975e46,bc9cc2a94,NaN,c5361037c,3.0,Grandmaster,Cold,h,C,OZ,5.0,12.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
599995,599995,0.0,1.0,0.0,T,N,Red,Polygon,Axolotl,India,Theremin,014770cf0,da5014b01,a7059911d,158183c63,015c63324,3.0,Novice,Freezing,a,R,GZ,5.0,NaN,0
599996,599996,1.0,0.0,0.0,T,Y,Blue,Polygon,Dog,Costa Rica,Oboe,NaN,2023ed4ed,83bdea3a5,e9fde8fa8,a02ae6a63,2.0,Novice,Boiling Hot,n,N,sf,NaN,3.0,0
599997,599997,0.0,0.0,0.0,F,Y,Red,Circle,Axolotl,Russia,Theremin,c7dc5d460,5d7d341ac,114b1dbf3,cccbca824,40f9610c1,2.0,Contributor,Freezing,n,H,MV,7.0,5.0,0
599998,599998,1.0,1.0,0.0,F,Y,NaN,Polygon,Axolotl,NaN,Piano,4d7780407,209e1054e,fba315672,4164322bd,c1a8374a0,1.0,Master,Warm,m,X,Ey,1.0,5.0,0


In [ ]:
def create_folds (df, split):
    # create a column
    df['kfold'] = -1

    # shuffle
    df = df.sample(frac = 1).reset_index(drop=True)

    # target 
    y = df.target.values

    #initiate kfold class
    kf= model_selection.StratifiedKFold(n_splits=split)

    #create folds
    for f_num, (r_old, r_new) in enumerate(kf.split(X = df, y= y)):
        df.loc[r_new,'kfold'] = int(f_num)
    
    return pd.DataFrame(df)

In [ ]:
df = create_folds(df,5)

In [ ]:
df.kfold.value_counts()

4    120000
3    120000
2    120000
1    120000
0    120000
Name: kfold, dtype: int64

In [ ]:
features = [f for f in df.columns if f not in ('id', 'kfold', 'target')]

In [ ]:
#Handle the missing values
for column in features:
    df.loc[:, column] = df[column].astype(str).fillna('NONE')

In [ ]:
df

,id,bin_0,bin_1,bin_2,bin_3,bin_4,nom_0,nom_1,nom_2,nom_3,nom_4,nom_5,nom_6,nom_7,nom_8,nom_9,ord_0,ord_1,ord_2,ord_3,ord_4,ord_5,day,month,target,kfold
0,511901,0.0,0.0,nan,F,N,Blue,Trapezoid,Dog,Finland,Theremin,a9f2c6001,b79f31595,0ffdb2c15,1a59581be,3124cd7b0,1.0,Master,Freezing,k,N,pE,1.0,12.0,0,0
1,495290,0.0,0.0,1.0,T,N,Blue,Trapezoid,Dog,India,Oboe,8f4ca911b,26bbdda34,857cbf8c7,b8e63cace,053cf1f9e,1.0,Grandmaster,Hot,d,A,EC,1.0,11.0,0,0
2,551555,0.0,0.0,0.0,T,Y,Red,Polygon,Hamster,Finland,Bassoon,ca14594c4,nan,623908296,90e9e961e,f8e5ca92b,2.0,Novice,Freezing,c,T,GZ,2.0,12.0,0,0
3,407143,0.0,0.0,1.0,F,Y,Red,Triangle,Lion,Canada,Theremin,a753f9fca,549b11326,fba315672,ef752c107,16644996e,3.0,Grandmaster,Cold,n,M,QV,1.0,7.0,1,0
4,75458,0.0,0.0,1.0,nan,Y,Red,Circle,Hamster,India,Theremin,92fb4c0dc,9dd923ce3,057ab40ea,nan,38c188a6a,3.0,Contributor,Cold,h,P,tn,3.0,8.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
599995,385036,0.0,0.0,0.0,F,N,Blue,Polygon,nan,Finland,Oboe,1ad522e08,759935ae8,5622ee17f,68f9b1267,cb3a45965,3.0,Contributor,Warm,o,X,nj,1.0,8.0,0,4
599996,102368,0.0,0.0,1.0,F,N,Red,Star,Axolotl,Costa Rica,Theremin,c926eedfd,bcd218228,ba9faf5b1,6eb21874b,bfc1e9aa0,2.0,Contributor,Freezing,n,F,cg,7.0,2.0,0,4
599997,2309,0.0,0.0,0.0,T,Y,Green,Triangle,Hamster,India,Piano,a6c427d48,598d91072,5d5b262b1,15f03b1f4,3fc84fbe9,1.0,Grandmaster,nan,h,B,HK,3.0,nan,0,4
599998,459041,0.0,0.0,0.0,T,Y,Green,Circle,Lion,Finland,Oboe,57ca6ec79,85d41f8c8,67a8d4ebb,d8bad19d0,f763f2031,1.0,Master,Warm,k,P,mP,6.0,nan,0,4


In [ ]:
#missing values handled
df.isnull().sum()

id        0
bin_0     0
bin_1     0
bin_2     0
bin_3     0
bin_4     0
nom_0     0
nom_1     0
nom_2     0
nom_3     0
nom_4     0
nom_5     0
nom_6     0
nom_7     0
nom_8     0
nom_9     0
ord_0     0
ord_1     0
ord_2     0
ord_3     0
ord_4     0
ord_5     0
day       0
month     0
target    0
kfold     0
dtype: int64

In [ ]:
# create training and validation sets
f_count = [0,1,2,2,4]
f_num = random.choice(f_count)

df_train = df[df.kfold != f_num].reset_index(drop=True)
df_valid = df[df.kfold == f_num].reset_index(drop=True)

In [ ]:
df_train

,id,bin_0,bin_1,bin_2,bin_3,bin_4,nom_0,nom_1,nom_2,nom_3,nom_4,nom_5,nom_6,nom_7,nom_8,nom_9,ord_0,ord_1,ord_2,ord_3,ord_4,ord_5,day,month,target,kfold
0,511901,0.0,0.0,nan,F,N,Blue,Trapezoid,Dog,Finland,Theremin,a9f2c6001,b79f31595,0ffdb2c15,1a59581be,3124cd7b0,1.0,Master,Freezing,k,N,pE,1.0,12.0,0,0
1,495290,0.0,0.0,1.0,T,N,Blue,Trapezoid,Dog,India,Oboe,8f4ca911b,26bbdda34,857cbf8c7,b8e63cace,053cf1f9e,1.0,Grandmaster,Hot,d,A,EC,1.0,11.0,0,0
2,551555,0.0,0.0,0.0,T,Y,Red,Polygon,Hamster,Finland,Bassoon,ca14594c4,nan,623908296,90e9e961e,f8e5ca92b,2.0,Novice,Freezing,c,T,GZ,2.0,12.0,0,0
3,407143,0.0,0.0,1.0,F,Y,Red,Triangle,Lion,Canada,Theremin,a753f9fca,549b11326,fba315672,ef752c107,16644996e,3.0,Grandmaster,Cold,n,M,QV,1.0,7.0,1,0
4,75458,0.0,0.0,1.0,nan,Y,Red,Circle,Hamster,India,Theremin,92fb4c0dc,9dd923ce3,057ab40ea,nan,38c188a6a,3.0,Contributor,Cold,h,P,tn,3.0,8.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
479995,385036,0.0,0.0,0.0,F,N,Blue,Polygon,nan,Finland,Oboe,1ad522e08,759935ae8,5622ee17f,68f9b1267,cb3a45965,3.0,Contributor,Warm,o,X,nj,1.0,8.0,0,4
479996,102368,0.0,0.0,1.0,F,N,Red,Star,Axolotl,Costa Rica,Theremin,c926eedfd,bcd218228,ba9faf5b1,6eb21874b,bfc1e9aa0,2.0,Contributor,Freezing,n,F,cg,7.0,2.0,0,4
479997,2309,0.0,0.0,0.0,T,Y,Green,Triangle,Hamster,India,Piano,a6c427d48,598d91072,5d5b262b1,15f03b1f4,3fc84fbe9,1.0,Grandmaster,nan,h,B,HK,3.0,nan,0,4
479998,459041,0.0,0.0,0.0,T,Y,Green,Circle,Lion,Finland,Oboe,57ca6ec79,85d41f8c8,67a8d4ebb,d8bad19d0,f763f2031,1.0,Master,Warm,k,P,mP,6.0,nan,0,4


In [ ]:
#one hot encoding
ohe = preprocessing.OneHotEncoder()
data = pd.concat([df_train[features], df_valid[features]], axis=0)
ohe.fit(data[features])

# transform training and validation data
x_train = ohe.transform(df_train[features])
x_valid = ohe.transform(df_valid[features])

In [ ]:
x_train.shape

(480000, 5723)

In [ ]:
#Use SVD
svd = decomposition.TruncatedSVD(n_components=120)
full_sparse = sparse.vstack((x_train, x_valid))
svd.fit(full_sparse)
x_train = svd.transform(x_train)
x_valid = svd.transform(x_valid)

In [ ]:
x_train.shape

(480000, 120)

In [ ]:
# Random forest
model = ensemble.RandomForestClassifier(n_jobs=-1)
model.fit(x_train, df_train.target.values)
valid_preds = model.predict_proba(x_valid)[:, 1]
auc = metrics.roc_auc_score(df_valid.target.values, valid_preds)
auc

0.7098161636757482